In [1]:
%load_ext lab_black

# Distributed training example using PyTorch

This notebook trains, evaluates and deploys a classifier for handwritten digits (Using the MNIST dataset). The training is distributed across multiple GPUs using DDP.


Author: Nick Lawrence ntl@us.ibm.com
License: [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0)

## Base container Image
The components of this pipeline use a a container image that has been built for this example. The Dockerfile is included in the [container_image](../../pytorch_model_frameworks/mnist/container_image) directory. 

The container image does *not* include a notebook server, meaning you cannot use the base image to launch a notebook. A new container image was created so that PyTorch 2.0 and Python 3.10 could be used in the example. These packages were not available when the notebook server's image was built. The image also includes pytorch-lightning, which makes designing a model simpler.


The disadvantage of using different images for pipeline steps is that it is more difficult to test code from within the notebook server itself, since not all packages are available.

The custom image also has installed the pytorch_distributed_kf_tools, which simplifies the creation and deployment of the PyTorch Job. The source code for this package is [here](../../distributed_kf_tools).

# Workspace

The pipeline allocates a shared PVC (the workspace) that is used to:
* Download python code for the model and training from github
* Load training and test data for the MNIST dataset
* Store checkpoints created during the training process
* Store tensorboard logs


## Imports and Constants

In [2]:
import kfp
from kfp.components import InputPath, OutputPath
import kfp.dsl as dsl
from kfp.dsl import PipelineConf, data_passing_methods
import numpy as np
import os
from typing import NamedTuple

BASE_IMAGE = "quay.io/ntlawrence/pytorch@sha256:75b0d6804e480a864ce2cc0e14be9b0bd993cc341dee346d29031defdba2f06d"
COMPONENT_CATALOG_FOLDER = f"{os.getenv('HOME')}/components"

# Prepare workspace component

This component performs several functions:
* download the training and test data to the workspace
* clone the python code for model and training from git
* create the directory for the tensorboard logs

In [3]:
def prepare_workspace():
    from torchvision.datasets import MNIST
    import subprocess
    import os

    # Download data
    _ = MNIST("/workspace/data", download=True, train=True)

    # Clone git repo
    subprocess.run(
        "git clone https://github.com/ntl-ibm/kubeflow-ppc64le-examples.git  /workspace/kubeflow-ppc64le-examples -b pytorch_example_improvements",
        shell=True,
    )

    os.mkdir("/workspace/tensorboard")


prepare_workspace_comp = kfp.components.create_component_from_func(
    prepare_workspace, base_image=BASE_IMAGE
)

In [4]:
def configure_tensorboard(
    mlpipeline_ui_metadata_path: OutputPath(),
    pvc_name: str,
    pvc_path: str = "",
    tensorboard_name: str = "",
) -> None:
    """
    Monitors a training job based on Tensorboard logs.
    Logs are expected to be written to the specified subpath of the pvc
    """
    from collections import namedtuple
    import json
    from kubernetes import client, config, watch
    import logging
    import sys
    import os
    import yaml
    import textwrap
    import json
    import http

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )
    logger = logging.getLogger()

    if not tensorboard_name:
        tensorboard_name = "{{workflow.name}}"

    namespace = "{{workflow.namespace}}"

    config.load_incluster_config()
    api_client = client.ApiClient()
    apps_api = client.AppsV1Api(api_client)
    custom_object_api = client.CustomObjectsApi(api_client)

    # Delete possible existing tensorboard
    try:
        custom_object_api.delete_namespaced_custom_object(
            group="tensorboard.kubeflow.org",
            version="v1alpha1",
            plural="tensorboards",
            namespace=namespace,
            name=tensorboard_name,
            body=client.V1DeleteOptions(),
        )
    except client.exceptions.ApiException as e:
        if e.status != http.HTTPStatus.NOT_FOUND:
            raise

    tensorboard_spec = textwrap.dedent(
        f"""\
            apiVersion: tensorboard.kubeflow.org/v1alpha1
            kind: Tensorboard
            metadata:
              name: "{tensorboard_name}"
              namespace: "{namespace}"
              ownerReferences:
                - apiVersion: v1
                  kind: Workflow
                  name: "{{workflow.name}}"
                  uid: "{{workflow.uid}}"
            spec:
              logspath: "pvc://{pvc_name}/{pvc_path}"
            """
    )

    logger.info(tensorboard_spec)

    custom_object_api.create_namespaced_custom_object(
        group="tensorboard.kubeflow.org",
        version="v1alpha1",
        plural="tensorboards",
        namespace=namespace,
        body=yaml.safe_load(tensorboard_spec),
        pretty=True,
    )

    tensorboard_watch = watch.Watch()
    try:
        for tensorboard_event in tensorboard_watch.stream(
            custom_object_api.list_namespaced_custom_object,
            group="tensorboard.kubeflow.org",
            version="v1alpha1",
            plural="tensorboards",
            namespace=namespace,
            field_selector=f"metadata.name={tensorboard_name}",
            timeout_seconds=0,
        ):

            logger.info(f"tensorboard_event: {json.dumps(tensorboard_event, indent=2)}")

            if tensorboard_event["type"] == "DELETED":
                raise RuntimeError("The tensorboard was deleted!")

            tensorboard = tensorboard_event["object"]

            if "status" not in tensorboard:
                continue

            deployment_state = "Progressing"
            if "conditions" in tensorboard["status"]:
                deployment_state = tensorboard["status"]["conditions"][-1][
                    "deploymentState"
                ]

            if deployment_state == "Progressing":
                logger.info("Tensorboard deployment is progressing...")
            elif deployment_state == "Available":
                logger.info("Tensorboard deployment is Available.")
                break
            elif deployment_state == "ReplicaFailure":
                raise RuntimeError(
                    "Tensorboard deployment failed with a ReplicaFailure!"
                )
            else:
                raise RuntimeError(f"Unknown deployment state: {deployment_state}")
    finally:
        tensorboard_watch.stop()

    button_style = (
        "align-items: center; "
        "appearance: none; "
        "background-color: rgb(26, 115, 232); "
        "border: 0px none rgb(255, 255, 255); "
        "border-radius: 3px; "
        "box-sizing: border-box; "
        "color: rgb(255, 255, 255); "
        "cursor: pointer; "
        "display: inline-flex; "
        "font-family: 'Google Sans', 'Helvetica Neue', sans-serif; "
        "font-size: 14px; "
        "font-stretch: 100%; "
        "font-style: normal; font-weight: 700; "
        "justify-content: center; "
        "letter-spacing: normal; "
        "line-height: 24.5px; "
        "margin: 0px 10px 2px 0px; "
        "min-height: 25px; "
        "min-width: 64px; "
        "padding: 2px 6px 2px 6px; "
        "position: relative; "
        "tab-size: 4; "
        "text-align: center; "
        "text-indent: 0px; "
        "text-rendering: auto; "
        "text-shadow: none; "
        "text-size-adjust: 100%; "
        "text-transform: none; "
        "user-select: none; "
        "vertical-align: middle; "
        "word-spacing: 0px; "
        "writing-mode: horizontal-tb;"
    )

    # See: https://github.com/kubeflow/kubeflow/blob/master/components/crud-web-apps/tensorboards/frontend/src/app/pages/index/index.component.ts
    # window.open(`/tensorboard/${tensorboard.namespace}/${tensorboard.name}/`);
    ui_address = f"/tensorboard/{namespace}/{tensorboard_name}/#scalars"

    markdown = textwrap.dedent(
        f"""\
        # Tensorboard
        - <a href="{ui_address}" style="{button_style}" target="_blank">Connect</a>
        - <a href="/_/tensorboards/" style="{button_style}" target="_blank">Manage all</a>
        """
    )

    markdown_output = {
        "type": "markdown",
        "storage": "inline",
        "source": markdown,
    }

    ui_metadata = {"outputs": [markdown_output]}
    with open(mlpipeline_ui_metadata_path, "w") as metadata_file:
        json.dump(ui_metadata, metadata_file)

    logging.info("Finished.")


configure_tensorboard_comp = kfp.components.create_component_from_func(
    func=configure_tensorboard,
    base_image=BASE_IMAGE,
    packages_to_install=["kubernetes"],
)

In [5]:
def train_model(shared_pvc_name: str, worker_image: str):

    import distributed_kf_tools.deploy as deploy
    from distributed_kf_tools.template import OwningWorkFlow, PvcMount
    import subprocess

    for retries in range(5):
        try:
            ## Start the PyTorch job for distributed training
            job_name = "{{workflow.name}}" + (f"-{retries:03d}" if retries else "")
            deploy.run_pytorch_job(
                # owning_workflow setups it up so that when the pipeline is deleted,
                # the training job is cleaned up
                owning_workflow=OwningWorkFlow(
                    name="{{workflow.name}}", uid="{{workflow.uid}}"
                ),
                # These place holders for namespace and job name are
                # filled in by Kubeflow when the pipeline runs.
                namespace="{{workflow.namespace}}",
                pytorch_job_name=job_name,
                # Shared volumes used by the training script
                pvcs=[
                    PvcMount(
                        pvc_name=(shared_pvc_name),
                        mount_path="/workspace",
                    )
                ],
                working_dir="/workspace/kubeflow-ppc64le-examples/distributed_training/pytorch_model_frameworks/mnist/src/",
                # The command to run in each worker
                # This starts with "torch.distributed.run" for DDP
                # The output model path needs to be stored on the pvc,
                # The output path for this POD points the tmp directory of THIS pod,
                # Storing it in the temp directory of the PyTorch worker pod doesn't work,
                # because this pod doesn't have access to that.
                command=[
                    "python",
                    "-m",
                    "torch.distributed.run",
                    "train.py",
                    "--root_dir=./work",
                    "--data_dir=/workspace/data",
                    "--model_ckpt=/workspace/models/best.ckpt",
                    "--batch_size=672",
                    "--checkpoint",
                    "--early_stopping",
                    "--max_epochs=50",
                    "--pytorchjob",
                    "--tensorboard=/workspace/tensorboard",
                ],
                # Number of workers (pods)
                num_workers=3,
                # Number of GPUs per worker (OK to leave this at 1)
                gpus_per_worker=1,
                # The base image used for the worker pods
                worker_image=worker_image,
            )

            break
        except RuntimeError as e:
            print(f"THE JOB FAILED BECAUSE OF ERROR {e}")


train_model_comp = kfp.components.create_component_from_func(
    train_model, base_image=BASE_IMAGE
)

In [6]:
def test_model() -> NamedTuple("EvaluationOutput", [("mlpipeline_metrics", "Metrics")]):
    import json
    from collections import namedtuple
    import subprocess

    subprocess.run(
        [
            "python",
            "test.py",
            "--root_dir=/tmp",
            "--data_dir=/workspace/data",
            "--model_ckpt=/workspace/models/best.ckpt",
            "--batch_size=672",
            "--metrics_json=/tmp/metrics.json",
        ],
        cwd="/workspace/kubeflow-ppc64le-examples/distributed_training/pytorch_model_frameworks/mnist/src/",
        check=True,
    )

    with open("/tmp/metrics.json", "r") as f:
        metrics = json.load(f)

    metrics = {
        "metrics": [
            {"name": "acc", "numberValue": metrics["test/acc"], "format": "RAW"},
            {"name": "F1", "numberValue": metrics["test/F1"], "format": "RAW"},
        ]
    }

    out_tuple = namedtuple("EvaluationOutput", ["mlpipeline_metrics"])
    return out_tuple(json.dumps(metrics))


test_model_comp = kfp.components.create_component_from_func(
    test_model, base_image=BASE_IMAGE
)

In [7]:
def convert_to_onnx(onnx_model: OutputPath(str)):
    import shutil
    import os
    from pathlib import Path
    import subprocess

    subprocess.run(
        [
            "python",
            "convert.py",
            "--root_dir=/tmp",
            "--data_dir=/workspace/data",
            "--model_ckpt=/workspace/models/best.ckpt",
            "--onnx=/tmp/model.onnx",
        ],
        cwd="/workspace/kubeflow-ppc64le-examples/distributed_training/pytorch_model_frameworks/mnist/src/",
        check=True,
    )

    directory = str(Path(onnx_model).parent.absolute())
    os.makedirs(directory, exist_ok=True)
    shutil.copy("/tmp/model.onnx", onnx_model)


convert_to_onnx_comp = kfp.components.create_component_from_func(
    convert_to_onnx, base_image=BASE_IMAGE
)

In [8]:
UPLOAD_MODEL_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/model-building/upload-model/component.yaml"
)

upload_model_comp = kfp.components.load_component_from_file(UPLOAD_MODEL_COMPONENT)

In [9]:
DEPLOY_INFERENCE_SERVICE_COMPONENT = f"{os.getenv('HOME')}/kubeflow-ppc64le-examples/deploy_triton_inference_service_component/deploy_triton_inference_service_component.yaml"

deploy_inference_service_comp = kfp.components.load_component_from_file(
    DEPLOY_INFERENCE_SERVICE_COMPONENT
)

In [10]:
@dsl.pipeline(
    name="Handwritten digit classification",
    description="An example pipeline that trains using distributed pytorch",
)
def mnist_pipeline():

    workspace_volume_volop = dsl.VolumeOp(
        name="Create workspace for training",
        resource_name="shared-workspace-pvc",
        modes=dsl.VOLUME_MODE_RWM,
        size="4Gi",
        set_owner_reference=True,
    )

    prepare_workspace_task = prepare_workspace_comp()
    prepare_workspace_task.add_pvolumes({"/workspace": workspace_volume_volop.volume})

    configure_tensorboard_task = configure_tensorboard_comp(
        pvc_name=workspace_volume_volop.volume.persistent_volume_claim.claim_name,
        pvc_path="tensorboard",
        tensorboard_name="tb-{{workflow.name}}",
    )
    configure_tensorboard_task.after(prepare_workspace_task)

    train_model_task = train_model_comp(
        workspace_volume_volop.volume.persistent_volume_claim.claim_name,
        worker_image=BASE_IMAGE,
    )
    train_model_task.after(prepare_workspace_task)
    train_model_task.after(configure_tensorboard_task)
    train_model_task.add_pvolumes({"/workspace": workspace_volume_volop.volume})

    test_model_task = test_model_comp()
    test_model_task.after(train_model_task)
    test_model_task.add_pvolumes({"/workspace": workspace_volume_volop.volume})

    convert_to_onnx_task = convert_to_onnx_comp()
    convert_to_onnx_task.after(test_model_task)
    convert_to_onnx_task.add_pvolumes({"/workspace": workspace_volume_volop.volume})

    model_name = "mnist"
    minio_url = "minio-service.kubeflow:9000"

    upload_model_task = upload_model_comp(
        convert_to_onnx_task.outputs["onnx_model"],
        minio_url=minio_url,
        export_bucket="{{workflow.namespace}}-models",
        model_format="onnx",
        model_name=model_name,
        model_version=1,
    )

    deploy_model_task = deploy_inference_service_comp(
        name=model_name,
        rm_existing=True,
        storage_uri="s3://{{workflow.namespace}}-models/onnx/",
        minio_url=minio_url,
        concurrency_target=4,  # soft limit, may be exceeded for short periods of time
        predictor_min_replicas=0,  # min_replicas supports scale to 0
        predictor_max_replicas=4,  # We don't want to scale till we consume all the available GPUs
        predictor_protocol="v2",
        # Uncomment these next two lines to use the GPU runtime and allocate GPUs
        # triton_runtime_version="21.08-py3-gpu",
        # predictor_gpu_allocation=1,  # gpu per replica
    )
    deploy_model_task.after(upload_model_task)

In [11]:
PIPELINE_NAME = "MNIST HW Classification Pipeline"

# Configure Pipeline

The first transformer disables caching for our pipeline. Kubeflow caches tasks based on input and output parameters. Because we are using shared storage, the input parameters are the same as previous runs, however the data on the shared storage is most likely different.

The second transformer adds a node constraint to all tasks. This is only needed in the IBM Lab. We currently have a few machines that are older Power 8 hardware in the cluster. Python 3.10 from RocketCE has been optimized for Power 9 and Power 10. We'll force all out pods to run on the newer AC922's with newer hardware. (This isn't an issue for environments that have all AC922s or newer.)

In [12]:
pipeline_conf = kfp.dsl.PipelineConf()

# Disable Caching
def disable_cache_transformer(op: dsl.ContainerOp):
    if isinstance(op, dsl.ContainerOp):
        op.execution_options.caching_strategy.max_cache_staleness = "P0D"
    else:
        op.add_pod_annotation(
            name="pipelines.kubeflow.org/max_cache_staleness", value="P0D"
        )
    return op


pipeline_conf.add_op_transformer(disable_cache_transformer)

# This transformer is only Relevant inside an IBM lab that has both P8 and P9 machines
# (Assumptioin is that the P9 machines have the ai.accelerator label on them)
def run_on_power_9_transformer(op: dsl.ContainerOp):
    if isinstance(op, dsl.ContainerOp):
        op.add_node_selector_constraint("ai.accelerator", "V100")


pipeline_conf.add_op_transformer(run_on_power_9_transformer)

# Compile, upload and run pipeline

This creates a run of the pipeline within an experiment.

The parameter to the pipeline is the name of the PVC to copy the model onto. *This PVC must have an access mode of ReadWriteMany!*


In [13]:
def delete_pipeline(pipeline_name: str):
    """Delete's a pipeline with the specified name"""

    client = kfp.Client()
    existing_pipelines = client.list_pipelines(page_size=999).pipelines
    matches = (
        [ep.id for ep in existing_pipelines if ep.name == pipeline_name]
        if existing_pipelines
        else []
    )
    for id in matches:
        client.delete_pipeline(id)


def get_experiment_id(experiment_name: str) -> str:
    """Returns the id for the experiment, creating the experiment if needed"""
    client = kfp.Client()
    existing_experiments = client.list_experiments(page_size=999).experiments
    matches = (
        [ex.id for ex in existing_experiments if ex.name == experiment_name]
        if existing_experiments
        else []
    )

    if matches:
        return matches[0]

    exp = client.create_experiment(experiment_name)
    return exp.id

In [14]:
PIPELINE_NAME = "train-mnist-pytorch-model"

client = kfp.Client()
kfp.compiler.Compiler().compile(
    pipeline_func=mnist_pipeline,
    package_path=f"{PIPELINE_NAME}.yaml",
    pipeline_conf=pipeline_conf,
)

delete_pipeline(PIPELINE_NAME)
uploaded_pipeline = client.upload_pipeline(f"{PIPELINE_NAME}.yaml", PIPELINE_NAME)
run = client.run_pipeline(
    experiment_id=get_experiment_id("mnist-pytorch-exp"),
    job_name="mnist-pytorch",
    pipeline_id=uploaded_pipeline.id,
    params={},
)

In [15]:
TWENTY_MIN = 20 * 60
result = client.wait_for_run_completion(run.id, timeout=TWENTY_MIN)
{
    "status": result.run.status,
    "error": result.run.error,
    "time": str(result.run.finished_at - result.run.created_at),
    "metrics": result.run.metrics,
}

{'status': 'Succeeded',
 'error': None,
 'time': '0:08:09',
 'metrics': [{'format': 'RAW',
   'name': 'acc',
   'node_id': 'handwritten-digit-classification-9kgcn-4184176891',
   'number_value': 0.9537000060081482},
  {'format': 'RAW',
   'name': 'F1',
   'node_id': 'handwritten-digit-classification-9kgcn-4184176891',
   'number_value': 0.9531377553939819}]}

# Make a prediction

In [58]:
from PIL import Image
import numpy as np

In [106]:
with Image.open("3.jpg") as i:
    i.show()
    image_array = np.asarray(i)

In [107]:
image_array.shape

(28, 28)

In [108]:
# channels dimension is required, even though the shape is always (1,28,28)
tensor = np.expand_dims(image_array, 0)

In [109]:
# batch dimension required
tensor = np.expand_dims(tensor, 0)
tensor.shape

(1, 1, 28, 28)

In [110]:
!oc get inferenceservice mnist

NAME    URL                                                          READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION             AGE
mnist   http://mnist-ntl-us-ibm-com.apps.openshift.ibmscoutsandbox   True           100                              mnist-predictor-default-00001   71m


In [111]:
import subprocess
import yaml

p = subprocess.run(
    "oc get inferenceservice mnist -o yaml", shell=True, stdout=subprocess.PIPE
)

desc = yaml.safe_load(p.stdout)
url = desc["status"]["address"]["url"]

In [112]:
requests.get(
    "http://mnist.ntl-us-ibm-com.svc.cluster.local/v2/models/mnist/versions/1"
).json()

{'name': 'mnist',
 'versions': ['1'],
 'platform': 'onnxruntime_onnx',
 'inputs': [{'name': 'input.1', 'datatype': 'FP32', 'shape': [1, 1, 28, 28]}],
 'outputs': [{'name': '18', 'datatype': 'FP32', 'shape': [1, 10]}]}

In [113]:
import requests

payload = {
    "inputs": [
        {
            "name": "input.1",
            "shape": list(tensor.shape),
            "datatype": "FP32",
            "data": tensor.flatten().tolist(),
        }
    ]
}

res = requests.post(url, headers={}, data=json.dumps(payload))

In [116]:
np.argmax(res.json()["outputs"][0]["data"])

3